In [4]:
import CIMS
import pickle
from datetime import datetime
import pandas as pd
import numpy as np

In [6]:
### Validate model ###

# Validator
time_start = datetime.now()

sheet_list = [
    # 'RunYears',
    'CIMS',
    'CAN',
    # 'BC',
    # 'AB',
    # 'SK',
    # 'MB',
    'ON',
    # 'QC',
    # 'AT',
]

year_list = [
    ### Historical ###
    2000,
    2005,
    2010,
    2015,
    2020,
    ### Forecast ###
    2025,
    # 2030,
    # 2035,
    # 2040,
    # 2045,
    # 2050,
]

col_list1 = [
    'Branch',
    'Sector',
    'Technology',
    'Parameter',
    'Context',
    'Sub_Context',
    'Target',
    'Source',
    'Unit',
]
col_list2 = pd.Series(np.arange(2000,2051,5), dtype='string').tolist()
col_list = col_list1 + col_list2

sector_list = [
    '',
    'Natural Gas Extraction',
]

# Base model
model_validator = CIMS.ModelValidator(
    infile='../models/CIMS_base model.xlsb',
    sheet_map = {
        'model' : sheet_list,
        'default_param' : 'Default values'},
    node_col = 'Branch',
    root_node='CIMS'
)
model_validator.validate(verbose=True, raise_warnings=False)

time_end = datetime.now()
print("Validator: ", time_end - time_start)
print("\n")


0 service name/branch mismatches. 
0 non-root nodes are never referenced. 
6 non-root nodes are never referenced. See ModelValidator.warnings['unreferenced_nodes'] for more info.
0 nodes were specified but don't provide a service. 
0 nodes had invalid competition types. 
0 nodes requested services of themselves. 
36 nodes or technologies don't request other services. See ModelValidator.warnings['nodes_no_requested_service'] for more info.
0 nodes have 0 in the output line. 
4 fuel nodes don't have a life cycle cost or price. See ModelValidator.warnings['fuels_without_lcc_or_price'] for more info.
0 technologies are missing a base year market share. 
0 nodes/technologies request from the same service more than once. 
46 nodes/technologies contain only 0's or are missing values in service request rows. See ModelValidator.warnings['bad_service_req'] for more info.
0 tech compete nodes don't contain Technology or Service headings. 
0 nodes/technologies requested services from nodes which a

In [7]:
model_validator.warnings['nodes_without_capital_cost']

KeyError: 'nodes_without_capital_cost'

In [ ]:
### Run model ###

import CIMS
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

time0_start = datetime.now()

#################### Base model ###########################

time_start = datetime.now()

sheet_list = [
    'CIMS',
    'CAN',
    # 'BC',
    # 'AB',
    # 'SK',
    # 'MB',
    'ON',
    # 'QC',
    # 'AT',
]

year_list = [
    ### Historical ###
    2000,
    2005,
    # 2010,
    # 2015,
    # 2020,
    ### Forecast ###
    # 2025,
    # 2030,
    # 2035,
    # 2040,
    # 2045,
    # 2050,
]

col_list1 = [
    'Branch',
    'Sector',
    'Technology',
    'Parameter',
    'Context',
    'Sub_Context',
    'Target',
    'Source',
    'Unit',
]
col_list2 = pd.Series(np.arange(2000,2051,5), dtype='string').tolist()
col_list = col_list1 + col_list2

# Use this sector list to calibrate individual sectors; comment out all entries if running model as normal
# Include 'None' to add CIMS and CAN when calibrating
# Must use Exogenous prices file below when calibrating
sector_list = [
    # None,
    # 'Natural Gas Extraction',
    # 'Petroleum Crude',
    # 'Mining',
    # 'Petroleum Refining',
    # 'Industrial Minerals',
    # 'Iron and Steel',
    # 'Metal Smelting',
    # 'Chemical Products',
    # 'Pulp and Paper',
    # 'Light Industrial',
    # 'Residential',
    # 'Commercial',
    # 'Transportation Personal',
    # 'Transportation Freight',
    # 'Waste',
    # 'Agriculture',
]

# Base model
model_reader = CIMS.ModelReader(infile='../models/CIMS_base model.xlsb',
                sheet_map = {'model' : sheet_list,
                            'default_param' : 'Default values'},
                node_col = 'Branch',
                col_list = col_list,
                year_list = year_list,
                sector_list = sector_list)
model = CIMS.Model(model_reader)

# Exogenous prices
model_reader = CIMS.ModelReader(infile='../models/CIMS_exogenous prices.xlsb',
                sheet_map = {'model' : sheet_list,
                            'default_param' : 'Default values'},
                node_col = 'Branch',
                col_list = col_list,
                year_list = year_list,
                sector_list = sector_list)
model = model.update(model_reader)

# # Declining capital costs
# model_reader = CIMS.ModelReader(infile='../models/CIMS_dcc.xlsb',
#                 sheet_map = {'model' : sheet_list,
#                             'default_param' : 'Default values'},
#                 node_col = 'Branch',
#                 col_list = col_list,
#                 year_list = year_list,
#                 sector_list = sector_list)
# model = model.update(model_reader)

# # Calibration
# model_reader = CIMS.ModelReader(infile='../models/CIMS_calibration.xlsb',
#                 sheet_map = {'model' : sheet_list,
#                             'default_param' : 'Default values'},
#                 node_col = 'Branch',
#                 col_list = col_list,
#                 year_list = year_list,
#                 sector_list = sector_list)
# model = model.update(model_reader)

time_end = datetime.now()
print("Base model: ", datetime.now() - time_start)
print("\n")

#################### Base model ###########################


#################### Reference scenario ###########################

ref_policies = [
    # 'Ref_CAN_carbon tax_OBPS.xlsb',
    # 'Ref_ON_coal phase out.xlsb',
    # 'Ref_CAN_ZEV.xlsb',
    # 'Ref_ON_renewable content transport fuels.xlsb',
    # 'Ref_ON_subsidies.xlsb',
    # 'Ref_ON_waste methane large sites.xlsb',
    ]

if ref_policies:
    time_start = datetime.now()

    for pol in ref_policies:
        pol_path = f"../models/reference policies/{pol}"
        model_reader = CIMS.ModelReader(infile=pol_path,
                    sheet_map = {'model' : sheet_list,
                            'default_param' : 'Default values'},
                    node_col = 'Branch',
                    col_list = col_list,
                    year_list = year_list,
                    sector_list = sector_list)
        model = model.update(model_reader)

    time_end = datetime.now()
    print("Reference scenario: ", datetime.now() - time_start)
    print("\n")

#################### Reference model ###########################


# Run model
time_start = datetime.now()
model.run(equilibrium_threshold=0.05, max_iterations=10, show_warnings=False, print_eq=False)
time_end = datetime.now()
print("Simulation time: ", time_end - time_start)
print("\n")

# Results
CIMS.log_model(model=model,
                output_file = '../results/results_general.csv',
                path = '../results/results_general.txt')

# CIMS.log_model(model=model,
#                   output_file='../results/results_DCC_stock.csv',
#                   path='../results/results_DCC_stock.txt')
#
# CIMS.log_model(model=model,
#                   output_file='../results/results_DCC.csv',
#                   path='../results/results_DCC.txt')


time0_end = datetime.now()
print("Total run time: ", time0_end - time0_start)
print("\n")